In [1]:
pip install git+https://github.com/keras-team/keras-preprocessing.git

  Cloning https://github.com/keras-team/keras-preprocessing.git to c:\users\user\appdata\local\temp\pip-req-build-es_giu__
  Resolved https://github.com/keras-team/keras-preprocessing.git to commit 3e380065d4afc7347aaee8d89325a16b22158438
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-preprocessing.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-es_giu__'


In [18]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [19]:
import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import layers

In [20]:
from tqdm import tqdm
from time import perf_counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

## 데이터 전처리

In [21]:
df = pd.read_csv("./data/FamilyData/Val/FamilyData_Val_DF.csv",index_col=0)

In [22]:
df.head()

,img_path,M,F,0's,10's,20's,30's,40's,50's,60+
0,F0801_IND_F_48_-45_01.JPG,1,0,0,0,0,0,1,0,0
1,F0801_IND_F_48_-45_02.JPG,1,0,0,0,0,0,1,0,0
2,F0801_IND_F_48_-90_01.JPG,1,0,0,0,0,0,1,0,0
3,F0801_IND_F_48_-90_02.JPG,1,0,0,0,0,0,1,0,0
4,F0801_IND_F_48_0_01.JPG,1,0,0,0,0,0,1,0,0


In [23]:
columns=["M","F","0's","10's","20's","30's","40's","50's","60+"]

In [24]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, random_state=777, test_size=0.2)
train_df, val_df = train_test_split(train_df, random_state=777, test_size=0.2)

len(train_df) , len(val_df), len(test_df)

(2048, 512, 640)

In [8]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [9]:
train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="C:/Users/user/Desktop/Preprocessed_Datasets(FamilyData_Val)",
    x_col="img_path",
    y_col=columns,
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="other",
    target_size=(224,224))

Found 2560 validated image filenames.


In [10]:
valid_generator=test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="C:/Users/user/Desktop/Preprocessed_Datasets(FamilyData_Val)",
    x_col="img_path",
    y_col=columns,
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="other",
    target_size=(224,224))

Found 320 validated image filenames.


In [11]:
test_generator=test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="C:/Users/user/Desktop/Preprocessed_Datasets(FamilyData_Val)",
    x_col="img_path",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode="other", #None
    target_size=(224,224))

Found 320 validated image filenames.


## 모델 전이학습

### 1) Import VGG16

In [12]:
#model
from keras.applications import VGG16

vgg16 = VGG16(weights='imagenet',
              include_top=False, 
              input_shape = (224,224,3))


In [13]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 2) Model Define

In [14]:
model = models.Sequential()
model.add(vgg16)

model.add(Flatten(input_shape=(224,224,3)))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(9,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 21154505 (80.70 MB)
Trainable params: 21154505 (80.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 3) Model Compile

In [15]:
model.compile(loss = 'binary_crossentropy',
              optimizer = optimizers.Adam(learning_rate = 0.00005),
              metrics = ['accuracy'])

### 4) Model Fit

In [17]:
%%time

Hist_dandc = model.fit(train_generator,
                        # steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        # validation_steps=STEP_SIZE_VALID,
                        epochs=20)

Epoch 1/20
80/80 [==============================] - 1221s 15s/step - loss: 0.3756 - accuracy: 0.7945 - val_loss: 0.2859 - val_accuracy: 0.9469
Epoch 2/20
27/80 [=========>....................] - ETA: 8:54 - loss: 0.2126 - accuracy: 0.9144

KeyboardInterrupt: 

In [ ]:
model.save('C:/Users/user/Desktop/mtvs/project4/modelmodels/vgg16.h5')

In [ ]:
print(Hist_dandc.history['val_loss'])
print(Hist_dandc.history['loss'])

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(Hist_dandc.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, Hist_dandc.history['loss'])
plt.plot(epochs, Hist_dandc.history['val_loss'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()